In [ ]:
from transformers import pipeline

In [ ]:
gen = pipeline("token-classification", "Isotonic/distilbert_finetuned_ai4privacy", device=-1)

In [ ]:
text = "My name is Clara and I live in Berkeley, California."

In [ ]:
output = gen(text, aggregation_strategy="simple")

In [ ]:
output

In [ ]:
def replace_entities(output, text):
    word_to_entity_group = dict(
    (text[token["start"] : token["end"]], token["entity_group"]) for token in output
)
    for i, token in enumerate(output):
        word = list(word_to_entity_group.keys())[i]
        text = text.replace(word, f"[{word_to_entity_group[word]}]")

    return text

In [ ]:
replace_entities(output, text)